In [1]:

#First, we set the CAMB parameters and results
import camb
from scipy import constants
from scipy import integrate
import numpy as np

class CAMBresults:

    def __init__(self, H0, ombh2, omch2, mnu, w0, wa, As, ns, kmax, redshifts):
        self.pars = camb.CAMBparams()
        self.kmax = kmax
        self.pars.set_cosmology(H0 = H0, ombh2 = ombh2, omch2 = omch2, mnu = mnu)
        self.pars.set_dark_energy(w = w0, wa = wa)
        self.pars.InitPower.set_params(As = As, ns = ns)
        self.pars.set_matter_power(redshifts = redshifts, kmax = kmax)
        self.results = camb.get_results(self.pars)

    def nonlinear_power(self, halofit_formula = 'takahashi'):
        self.pars.NonLinear = camb.model.NonLinear_both
        self.pars.NonLinearModel.set_params(halofit_version=halofit_formula)

    def power_spec_z0(self):
        self.results.calc_power_spectra(self.pars)
        values = self.results.get_matter_power_spectrum(minkh = 1e-4, maxkh = self.kmax, npoints = 500)
        self.kh, self.z, self.pk = values
        self.PK = camb.get_matter_power_interpolator(self.pars, hubble_units = True, k_hunit = True, log_interp = False)


In [5]:
class Hubble:
    """Obtain Hubble function
        Parameters:
        H0: Hubble constant
    """
    def __init__(self, H0, h_z):
        self.H0 = H0
        self._h_z = h_z

    def E_z(self, z):
        """E(z)
        Parameters:
        z: Redshift
        Notes: H(z) in km/s/Mpc while CAMB's H(z) is in 1/Mpc
        """
        h_z_phys = self._h_z*constants.c
        return h_z_phys / self.H0

prueba = Hubble(0.67, 10)
print('prueba:', prueba.E_z(2.0))

prueba: 4474514298.5074625


In [6]:
class ComovingDistance(Hubble):
    """Comvoing Distance
    Parameters:
    H0: Hubble constant"""
    
    def __init__(self, H0, h_z):
        super(ComovingDistance, self).__init__(H0, h_z)
        self.r_z_vec = np.vectorize(self.rcom)

    def rcom_prime(self, z):
        return self.r_z_vec(z)/(constants.c/self.H0)

    def drdz(self, z):
        return (constants.c/self.H0)/self.E_z(z)

    def rcom(self, z):
        return integrate.romberg(self.drdz, 0., z)

prueba1 = ComovingDistance(0.67, 10)
print('prueba1:', prueba1.rcom_prime(4))

prueba1: 8.939517751310475e-10


In [ ]:
class growthFactor(Hubble):
    """Growth Factor
        Parameters:
            Omega_0:  
            z: redshift
            Gamma: 
    """
    def __init__(self, omega0, z, gamma, H0, h_z):
        super(growthFactor, self).__init__(H0, h_z)
        self.growthFunction = ((omega0*((1 + z)**3))/self.E_z(z))**gamma
        
        """
        self.omega0 = _omega0
        self.z = _z
        self.gamma = _gamma
        
    def growthFunction(self, omega0, z, gamma):
        return ((omega0*((1 + z)**3))/self.E_z(z))**gamma
    """
    
    def dgf_dz(self, z):
        return self.growthFunction/(1 + z)
    
    def Growth_Factor(self, z):
        return np.exp(integrate.romberg(self.dgf_dz(z), 0., z))

prueba2 = growthFactor(0.8, 2, 0.3, 0.67, 10)
print(prueba2.Growth_Factor(2))

In [ ]:
class Temperature(Hubble): #obtaining the temperature (T^2(z) for the Power Spectrum (PHI(z))
    def omega_HI(self, densityHI, z):
        return (8*np.pi*sp.G*densityHI)/(3*(Hubble.H0**2))
    
    def Temp_obs(self, z):
        return 44e-6*(Temperature.omega_HI*(Hubble.H0/100)/2.45e-4)*((1 + z)**2)/Hubble.E_z

In [ ]:
class PowerSpectrum(): #Obtain the Power Spectrum (PHI(z))
    def bias(self, z):
        return

    def Power_Spectrum(self, k, z):
        return Temperature.Temp_obs
